In [2]:
%%time
import sys
sys.path.insert(0, '/Users/shuyafeng/PycharmProjects/xiaozaizi/codes/')

import os
import json
import warnings
import pprint


%reload_ext autoreload
%autoreload
%autoreload 2
%config Completer.use_jedi = False

CPU times: user 17.9 ms, sys: 15.4 ms, total: 33.3 ms
Wall time: 32.6 ms


In [3]:
import numpy as np
import scipy.io
from numpy.random import choice, laplace
from scipy.special import lambertw
from scipy.stats import laplace
from codes.kse_test import kse_test
from codes.dbscan import dbscan
from codes.forest import forest
from codes.tree import tree
import pandas as pd
import random


In [33]:
dataset4 = scipy.io.loadmat('/Users/shuyafeng/PycharmProjects/xiaozaizi/4.mat')['dataset4']
sizeofD = 100
dataset2 = dataset4[:, :3]
S, _ = np.histogram(dataset2, bins=sizeofD)
eps = [0.5,1,2,3,4,5,6,7,8,9,10]
B = S.reshape(-1, 1)
C = B[:sizeofD]

Output = tree(C)

pp = 1. / (Output ** 5)
pp = pp.reshape(-1)
pp /= pp.sum()
NN = 2 ** (15 * Output)
px = np.arange(1, sizeofD+1)

gamma = 0.5
rho = np.exp(lambertw(-gamma / (2 * np.exp(0.5)), k=-1) + 0.5)
rho = np.real(rho)
m = np.log(1 / rho) / (2 * (gamma - rho) ** 2)
k = m * (1 - gamma + rho + np.sqrt(np.log(1 / rho) / (2 * m)))
m = int(float(np.ceil(m)))
k = int(float(np.ceil(k)))


In [41]:
EVAL = np.zeros((len(eps), 7))
#EVAL1 = np.zeros((4, 7))

for rr in range(len(eps)):
    for j in range(1):
        C = B[:sizeofD].reshape(-1)
        GS = np.sort(np.random.choice(C, size=m, replace=False))
        Sensunif = GS[k-1]
        SS = np.sort(np.random.choice(px, size=m, replace=True, p=pp))
        SS = NN[np.floor(SS).astype(int)]
        SS = np.sort(SS)
        Sensanom = SS[k-1]
        C= B[:sizeofD]
        #unif = laplace.rvs(loc=0, scale=Sensunif/(eps[rr]), size=sizeofD) + C
        unif=C
        unif=[i + laplace.rvs(loc=0, scale=Sensunif/(eps[rr]),size=1) for i in unif]

        #noiseanom = laplace.rvs(loc=0, scale=Sensanom/(eps[rr]), size=sizeofD) + NN
        noiseanom = NN
        noiseanom=[i + laplace.rvs(loc=0, scale=Sensanom/(eps[rr]),size=1) for i in noiseanom]
        
        
        #noiseanom[noiseanom<0]=0
        noiseunif = tree(unif)
        noiseanom = tree(noiseanom)

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if Output[i] > 0.8:
                Index[i, 1] = 1

        ACTUAL = Index
        Origindex = np.where(ACTUAL[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseunif[i] > 0.8:
                Index[i, 1] = 1

        Uniformm = Index
        Pertindex = np.where(Uniformm[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseanom[i] > 0.8:
                Index[i, 1] = 1

        Anomm = Index
        Pertindex1 = np.where(Anomm[:, 1] == 1)[0]
        IND1 = Anomm
        IND = Uniformm

        C = len(set(Origindex) | set(Pertindex))
        p = len(Origindex)
        n = sizeofD - p
        N = p + n
        tp = sum(ACTUAL[Origindex, 1] == IND[Origindex, 1])
        tn = N - C
        fp = n - tn
        fn = p - tp
        tp_rate = tp / p
        tn_rate = tn / n
        accuracy = (tp + tn) / N
        sensitivity = tp_rate
        specificity = tn_rate
        precision = tp / (tp + fp)
        recall = sensitivity
        f_measure = 2 * ((precision * recall) / (precision + recall))
        gmean = np.sqrt(tp_rate * tn)
        EVAL[rr, :] += np.array([accuracy, tp, specificity, precision, recall, f_measure, gmean])



In [36]:
dataset4 = np.array(pd.read_csv('tr_server_data.csv'))
sizeofD = 100
dataset2 = dataset4[:, :3]
S, _ = np.histogram(dataset2, bins=sizeofD)
eps = [0.5,1,2,3,4,5,6,7,8,9,10]
B = S.reshape(-1, 1)
C = B[:sizeofD]

Output = tree(C)

pp = 1. / (Output ** 5)
pp = pp.reshape(-1)
pp /= pp.sum()
NN = 2 ** (15 * Output)
px = np.arange(1, sizeofD+1)

gamma = 0.5
rho = np.exp(lambertw(-gamma / (2 * np.exp(0.5)), k=-1) + 0.5)
rho = np.real(rho)
m = np.log(1 / rho) / (2 * (gamma - rho) ** 2)
k = m * (1 - gamma + rho + np.sqrt(np.log(1 / rho) / (2 * m)))
m = int(float(np.ceil(m)))
k = int(float(np.ceil(k)))


In [37]:
EVAL1 = np.zeros((len(eps), 7))
#EVAL1 = np.zeros((4, 7))

for rr in range(len(eps)):
    for j in range(1):
        C = B[:sizeofD].reshape(-1)
        GS = np.sort(np.random.choice(C, size=m, replace=False))
        Sensunif = GS[k-1]
        SS = np.sort(np.random.choice(px, size=m, replace=True, p=pp))
        SS = NN[np.floor(SS).astype(int)]
        SS = np.sort(SS)
        Sensanom = SS[k-1]
        C= B[:sizeofD]
        #unif = laplace.rvs(loc=0, scale=Sensunif/(eps[rr]), size=sizeofD) + C
        unif=C
        unif=[i + laplace.rvs(loc=0, scale=Sensunif/(eps[rr]),size=1) for i in unif]

        #noiseanom = laplace.rvs(loc=0, scale=Sensanom/(eps[rr]), size=sizeofD) + NN
        noiseanom = NN
        noiseanom=[i + laplace.rvs(loc=0, scale=Sensanom/(eps[rr]),size=1) for i in noiseanom]
        
        
        #noiseanom[noiseanom<0]=0
        noiseunif = tree(unif)
        noiseanom = tree(noiseanom)

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if Output[i] > 0.8:
                Index[i, 1] = 1

        ACTUAL = Index
        Origindex = np.where(ACTUAL[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseunif[i] > 0.8:
                Index[i, 1] = 1

        Uniformm = Index
        Pertindex = np.where(Uniformm[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseanom[i] > 0.8:
                Index[i, 1] = 1

        Anomm = Index
        Pertindex1 = np.where(Anomm[:, 1] == 1)[0]
        IND1 = Anomm
        IND = Uniformm

        C = len(set(Origindex) | set(Pertindex))
        p = len(Origindex)
        n = sizeofD - p
        N = p + n
        tp = sum(ACTUAL[Origindex, 1] == IND[Origindex, 1])
        tn = N - C
        fp = n - tn
        fn = p - tp
        tp_rate = tp / p
        tn_rate = tn / n
        accuracy = (tp + tn) / N
        sensitivity = tp_rate
        specificity = tn_rate
        precision = tp / (tp + fp)
        recall = sensitivity
        f_measure = 2 * ((precision * recall) / (precision + recall))
        gmean = np.sqrt(tp_rate * tn)
        EVAL1[rr, :] += np.array([accuracy, tp, specificity, precision, recall, f_measure, gmean])



## baseline

In [60]:
dataset4 = scipy.io.loadmat('/Users/shuyafeng/PycharmProjects/xiaozaizi/4.mat')['dataset4']
#dataset4 = np.array(pd.read_csv('tr_server_data.csv'))
sizeofD = 100
dataset2 = dataset4[:, :1]
S, _ = np.histogram(dataset2, bins=sizeofD)
eps = [0.5,1,2,3,4,5,6,7,8,9,10]
B = S.reshape(-1, 1)
C = B[:sizeofD]
Output = tree(C)

pp = 1. / (Output ** 5)
pp = pp.reshape(-1)
pp /= pp.sum()
NN = 2 ** (15 * Output)
px = np.arange(1, sizeofD+1)

gamma = 0.5
rho = np.exp(lambertw(-gamma / (2 * np.exp(0.5)), k=-1) + 0.5)
rho = np.real(rho)
m = np.log(1 / rho) / (2 * (gamma - rho) ** 2)
k = m * (1 - gamma + rho + np.sqrt(np.log(1 / rho) / (2 * m)))
m = int(float(np.ceil(m)))
k = int(float(np.ceil(k)))


In [64]:
#EVAL = np.zeros((4, 7))
EVAL2 = np.zeros((len(eps), 7))

for rr in range(len(eps)):
    for j in range(1):
        C = B[:sizeofD].reshape(-1)
        
        GS = np.sort(np.random.choice(C, size=m, replace=False))
        Sensunif = GS[k-1]
        SS = np.sort(np.random.choice(px, size=m, replace=False))
        SS = NN[np.floor(SS).astype(int)]
        SS = np.sort(SS)
        Sensanom = SS[k-1]

        C= B[:sizeofD]
        #unif = laplace.rvs(loc=0, scale=Sensunif/(eps[rr]), size=sizeofD) + C
        unif=C
        unif=[i + laplace.rvs(loc=0, scale=Sensunif/(eps[rr]),size=1) for i in unif]

        #noiseanom = laplace.rvs(loc=0, scale=Sensanom/(eps[rr]), size=sizeofD) + NN
        noiseanom = NN
        noiseanom=[i + laplace.rvs(loc=0, scale=Sensanom/(eps[rr]),size=1) for i in noiseanom]
        

        noiseunif = tree(unif)
        noiseanom = tree(noiseanom)

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if Output[i] > 0.8:
                Index[i, 1] = 1

        ACTUAL = Index
        Origindex = np.where(ACTUAL[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseunif[i] > 0.8:
                Index[i, 1] = 1

        Uniformm = Index
        Pertindex = np.where(Uniformm[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseanom[i] > 0.8:
                Index[i, 1] = 1

        Anomm = Index
        Pertindex1 = np.where(Anomm[:, 1] == 1)[0]
        IND1 = Anomm
        IND = Uniformm

        C = len(set(Origindex) | set(Pertindex))
        p = len(Origindex)
        n = sizeofD - p
        N = p + n
        tp = sum(ACTUAL[Origindex, 1] == IND[Origindex, 1])
        tn = N - C
        fp = n - tn
        fn = p - tp
        tp_rate = tp / p
        tn_rate = tn / n
        accuracy = (tp + tn) / N
        sensitivity = tp_rate
        specificity = tn_rate
        precision = tp / (tp + fp)
        recall = sensitivity
        f_measure = 2 * ((precision * recall) / (precision + recall))
        gmean = np.sqrt(tp_rate * tn)
        EVAL2[rr, :] += np.array([accuracy, tp, specificity, precision, recall, f_measure, gmean])



In [65]:
#dataset4 = scipy.io.loadmat('/Users/shuyafeng/PycharmProjects/xiaozaizi/server1.mat')['dataset4']
dataset4 = np.array(pd.read_csv('tr_server_data.csv'))
sizeofD = 100
dataset2 = dataset4[:, :1]
S, _ = np.histogram(dataset2, bins=sizeofD)
eps = [0.5,1,2,3,4,5,6,7,8,9,10]
B = S.reshape(-1, 1)
C = B[:sizeofD]
Output = tree(C)

pp = 1. / (Output ** 5)
pp = pp.reshape(-1)
pp /= pp.sum()
NN = 2 ** (15 * Output)
px = np.arange(1, sizeofD+1)

gamma = 0.5
rho = np.exp(lambertw(-gamma / (2 * np.exp(0.5)), k=-1) + 0.5)
rho = np.real(rho)
m = np.log(1 / rho) / (2 * (gamma - rho) ** 2)
k = m * (1 - gamma + rho + np.sqrt(np.log(1 / rho) / (2 * m)))
m = int(float(np.ceil(m)))
k = int(float(np.ceil(k)))


In [67]:
#EVAL = np.zeros((4, 7))
EVAL3 = np.zeros((len(eps), 7))

for rr in range(len(eps)):
    for j in range(1):
        C = B[:sizeofD].reshape(-1)
        
        GS = np.sort(np.random.choice(C, size=m, replace=False))
        Sensunif = GS[k-1]
        SS = np.sort(np.random.choice(px, size=m, replace=False))
        SS = NN[np.floor(SS).astype(int)]
        SS = np.sort(SS)
        Sensanom = SS[k-1]

        C= B[:sizeofD]
        #unif = laplace.rvs(loc=0, scale=Sensunif/(eps[rr]), size=sizeofD) + C
        unif=C
        unif=[i + laplace.rvs(loc=0, scale=Sensunif/(eps[rr]),size=1) for i in unif]

        #noiseanom = laplace.rvs(loc=0, scale=Sensanom/(eps[rr]), size=sizeofD) + NN
        noiseanom = NN
        noiseanom=[i + laplace.rvs(loc=0, scale=Sensanom/(eps[rr]),size=1) for i in noiseanom]
        

        noiseunif = tree(unif)
        noiseanom = tree(noiseanom)

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if Output[i] > 0.8:
                Index[i, 1] = 1

        ACTUAL = Index
        Origindex = np.where(ACTUAL[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseunif[i] > 0.8:
                Index[i, 1] = 1

        Uniformm = Index
        Pertindex = np.where(Uniformm[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseanom[i] > 0.8:
                Index[i, 1] = 1

        Anomm = Index
        Pertindex1 = np.where(Anomm[:, 1] == 1)[0]
        IND1 = Anomm
        IND = Uniformm

        C = len(set(Origindex) | set(Pertindex))
        p = len(Origindex)
        n = sizeofD - p
        N = p + n
        tp = sum(ACTUAL[Origindex, 1] == IND[Origindex, 1])
        tn = N - C
        fp = n - tn
        fn = p - tp
        tp_rate = tp / p
        tn_rate = tn / n
        accuracy = (tp + tn) / N
        sensitivity = tp_rate
        specificity = tn_rate
        precision = tp / (tp + fp)
        recall = sensitivity
        f_measure = 2 * ((precision * recall) / (precision + recall))
        gmean = np.sqrt(tp_rate * tn)
        EVAL3[rr, :] += np.array([accuracy, tp, specificity, precision, recall, f_measure, gmean])



In [68]:
m=pd.DataFrame(EVAL, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

In [56]:
m1=pd.DataFrame(EVAL1, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

In [69]:
m2=pd.DataFrame(EVAL2, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

In [70]:
m3=pd.DataFrame(EVAL3, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

In [71]:
to_save=[m['accuracy'],m['precision'],m['recall'],m['f_measure'],m1['accuracy'],m1['precision'],m1['recall'],m1['f_measure'],m2['accuracy'],m2['precision'],m2['recall'],m2['f_measure'],m3['accuracy'],m3['precision'],m3['recall'],m3['f_measure']]
to_save=pd.DataFrame(to_save).T
to_save.columns=['accuracy','precision','recall','f_measure','accuracy1','precision1','recall1','f_measure1','accuracy2','precision2','recall2','f_measure2','accuracy3','precision3','recall3','f_measure3']
to_save.to_csv('tree1.csv', index=False)


In [42]:
pd.DataFrame(EVAL, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

,accuracy,tp,specificity,precision,recall,f_measure,gmean
0,0.59,2.0,0.770270,0.105263,0.076923,0.088889,2.093947
1,0.65,5.0,0.810811,0.263158,0.192308,0.222222,3.396831
2,0.94,24.0,0.945946,0.857143,0.923077,0.888889,8.038370
3,0.76,18.0,0.783784,0.529412,0.692308,0.600000,6.336706
4,0.77,10.0,0.905405,0.588235,0.384615,0.465116,5.076340
5,0.73,12.0,0.824324,0.480000,0.461538,0.470588,5.306020
6,0.81,15.0,0.891892,0.652174,0.576923,0.612245,6.170650
7,0.76,15.0,0.824324,0.535714,0.576923,0.555556,5.932310
8,0.74,11.0,0.851351,0.500000,0.423077,0.458333,5.162736
9,0.82,18.0,0.864865,0.642857,0.692308,0.666667,6.656402


In [43]:
pd.DataFrame(EVAL1, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

,accuracy,tp,specificity,precision,recall,f_measure,gmean
0,0.59,2.0,0.770270,0.105263,0.076923,0.088889,2.093947
1,0.68,7.0,0.824324,0.350000,0.269231,0.304348,4.052540
2,0.68,7.0,0.824324,0.350000,0.269231,0.304348,4.052540
3,0.69,10.0,0.797297,0.400000,0.384615,0.392157,4.763644
4,0.71,12.0,0.797297,0.444444,0.461538,0.452830,5.218311
5,0.81,15.0,0.891892,0.652174,0.576923,0.612245,6.170650
6,0.81,20.0,0.824324,0.606061,0.769231,0.677966,6.850042
7,0.77,19.0,0.783784,0.542857,0.730769,0.622951,6.510347
8,0.97,26.0,0.959459,0.896552,1.000000,0.945455,8.426150
9,0.83,19.0,0.864865,0.655172,0.730769,0.690909,6.838803


In [45]:
m=pd.DataFrame(EVAL, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

In [46]:
m1=pd.DataFrame(EVAL1, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

In [47]:
to_save=[m['accuracy'],m['precision'],m['recall'],m['f_measure'],m1['accuracy'],m1['precision'],m1['recall'],m1['f_measure']]
to_save=pd.DataFrame(to_save).T
to_save.columns=['accuracy','precision','recall','f_measure','accuracy1','precision1','recall1','f_measure1',]
to_save.to_csv('tree.csv', index=False)